## Import

In [9]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import numpy as np
import pandas as pd
import random
import re

import seedir as sd

import time
import datetime
from datetime import datetime

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [47]:
# создаем папку для хранения результатов
sd.seedir(style='emoji', itemlimit=10, beyond='ellipsis', exclude_folders=['.git', '.ipynb_checkpoints'])

📁 HW 3/
├─📁 output/
│ └─📄 df_appeals.csv
├─📁 screenshots/
│ ├─📄 AppealPage.png
│ └─📄 MainPageWithRef.png
├─📁 selenium_chromedriver/
│ └─📄 chromedriver.exe
└─📄 SetelemAppealsParserBankiRu.ipynb


---
## Содержание

-[Описание задачи](#1)   
-[Тест: парсинг одного отзыва](#2)  
-[Реализация с requests](#3)  
-[Реализация с selenium](#4)  
-[Анализ данных](#5)  

---
<a name="1"></a>

## Описание задачи

- Сайт [Banki.ru](https://www.banki.ru/) - финансовый маркетплейс, на нем реализован один из крупнейших в РФ агрегатор отзывов о работе различных банков  
- Соберем с него отзывы о банке Сетелем ([страница с отзывами с оценкой 3/5](https://www.banki.ru/services/responses/bank/bnpparibaseast/?rate=3&is_countable=on&page=1&isMobile=0))  
- Для ускорения процесса будем собирать по 3 страницы отзывов с каждой оценкой (1-5)  
- [Пример отзыва с оценкой 3/5](https://www.banki.ru/services/responses/bank/response/10598288/)
- Со сраницы с отзывом будем собирать следующую информацию:  
    + Тема обращения
    + Время обращения
    + Автор обращения
    + Регион атора обращения
    + Текст обращения
    + Автор ответа
    + Время ответа
    + Текст ответа

![Страница отзыва с необходимой информацией](screenshots/AppealPage.png)

---
<a name="2"></a>

## Тест: парсинг одного отзыва

In [3]:
# попробуем собрать данные по приведенному выше отзыву
# страница с отзывами с оценкой 3/5
url = 'https://www.banki.ru/services/responses/bank/bnpparibaseast/?rate=3&is_countable=on&page=1&isMobile=0'

r = requests.get(url)
s = BeautifulSoup(r.text, 'lxml')

![Страница с отзывами](screenshots/MainPageWithRef.png)

In [4]:
# на теме обращения ссылка с номером обращения, по которому можно перейти на страницу с отзывом
# собираем все номера обращений
appeal_numbers = re.findall('/services/responses/bank/response/(\d*)', r.text)
appeal_numbers = list(dict.fromkeys(appeal_numbers))
print(f'Всего ссылок на обращения на 1 странице: {len(appeal_numbers)}')
print(f'Номер первого обращения: {appeal_numbers[0]}')
print(f'Номер последнего обращения: {appeal_numbers[-1]}')

Всего ссылок на обращения на 1 странице: 25
Номер первого обращения: 10598288
Номер последнего обращения: 10137465


In [4]:
# переходим на страницу конкретного отзыва
# и собираем необходимую информацию из html-структуры
i = 0
r_appeal = requests.get(f'https://www.banki.ru/services/responses/bank/response/{appeal_numbers[i]}/')
s_appeal = BeautifulSoup(r_appeal.text, 'lxml')

In [5]:
# Тема обращения
theme_appeal = s_appeal.findAll('h1', attrs={'class': 'text-header-0 le856f50c'})[0].get_text(strip=True)
theme_appeal

'Получение автокредита'

In [6]:
# Текст обращения 
text_appeal = s_appeal.findAll('div', attrs={'class': 'lb1789875'})[0].get_text(strip=True)
text_appeal

'Присматривал автомобиль в автосалоне. Были планы брать за наличку, но менеджер уговорил на автокредит - по нему можно скидку сделать, так что подешевле будет.В принципе схема понятная, так что я согласился. Подсел к кредитному менеджеру и тут пошли сказки мне рассказывать. Все плюсы рассказали, про минусы умолчали. В общем, всё как всегда. Дальше пошло от откровенное враньё. Что страховку можно через несколько месяцев частично вернуть (как потом оказалось, в договоре страхования чётко прописано, что страховая премия в таких случаях не возвращается), что досрочно погасить можно не ранее, чем через 3 месяца, а первый досрочный платёж можно внести не ранее чем через 1,5 месяца.В общем взял я этот кредит, но верить не стал. Почитал законы, позвонил в контактный центр - гасить можно любыми сумами хоть сегодня, страховку можно вернуть только в течении 14 дней. Причём в по телефону в контактном центре ничего не скрывают, рассказывают и даже сами предлагают оформить заявки на досрочное погаше

In [7]:
# Время обращения 
time_appeal = s_appeal.findAll('span', attrs={'class': 'l10fac986'})[0].get_text(strip=True)
time_appeal

'10.12.2021 13:34'

In [8]:
# Автор обращения 
user_appeal = s_appeal.findAll('span', attrs={'class': 'l17191939'})[0].get_text(strip=True)
user_appeal

'Andrey10699'

In [9]:
# Регион автора обращения 
por_appeal = s_appeal.findAll('span', attrs={'class': 'l3a372298'})[0].get_text(strip=True)
por_appeal

'Старый Оскол (Белгородская область)'

In [10]:
# Текст ответа
text_reply = s_appeal.findAll('div', attrs={'class': 'lb1789875'})[1].get_text(strip=True)
text_reply

'Добрый день.После получения Вашего отзыва мы получили комментарии относительно изложенной Вами ситуации. Нам стало известно, что при обсуждении покупки автомобиля в кредит сотрудник Дилерского центра консультировал Вас по условиям кредитования, к сожалению, провести проверку консультации сотрудника сторонней организации у нас нет возможности.Однако далее, Вас пригласили для оформления кредита к сотруднику Банка, который довел до Вашего сведения полную и подробную информация об условиях кредита, в том числе озвучив информация о возможности осуществления полного/частичного досрочного погашения кредита с любого срока. Также сотрудник проинформировал Вас согласно памятке к Договору страхования , что «страхователь в течение срока действия Договора страхования, заключенного в целях обеспечения исполнения обязательств заемщика по Договору потребительского кредита(займа), при условии полного досрочного исполнения Заемщиком(страхователем) обязательств по Договору имеет право отказаться от Дого

In [11]:
# Время ответа
time_reply = s_appeal.findAll('div', attrs={'class': 'l46c44745'})[0].get_text(strip=True)
time_reply

'2021-12-13 12:56:21'

In [12]:
# Автор ответа
user_reply = s_appeal.findAll('div', attrs={'class': 'l5b3cd260'})[0].get_text(strip=True)
user_reply

'Сетелем Банк'

---
<a name="3"></a>

## Реализация с requests

In [22]:
def exception_handler(func):
    '''
        Декоратор для возвращения 'н/д' в случае ошибки
    '''
    def inner_function(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            return 'н/д'
    return inner_function

# функции для сбора информации выше 
@exception_handler
def get_theme_appeal(s_appeal):
    return s_appeal.findAll('h1', attrs={'class': 'text-header-0 le856f50c'})[0].get_text(strip=True)

@exception_handler
def get_text_appeal(s_appeal):
    return s_appeal.findAll('div', attrs={'class': 'lb1789875'})[0].get_text(strip=True)

@exception_handler
def get_time_appeal(s_appeal):
    return s_appeal.findAll('span', attrs={'class': 'l10fac986'})[0].get_text(strip=True)

@exception_handler
def get_user_appeal(s_appeal):
    return s_appeal.findAll('span', attrs={'class': 'l17191939'})[0].get_text(strip=True)

@exception_handler
def get_por_appeal(s_appeal):
    return s_appeal.findAll('span', attrs={'class': 'l3a372298'})[0].get_text(strip=True)

@exception_handler
def get_text_reply(s_appeal):
    return s_appeal.findAll('div', attrs={'class': 'lb1789875'})[1].get_text(strip=True)

@exception_handler
def get_time_reply(s_appeal):
    return s_appeal.findAll('div', attrs={'class': 'l46c44745'})[0].get_text(strip=True)

@exception_handler
def get_user_reply(s_appeal):
    return s_appeal.findAll('div', attrs={'class': 'l5b3cd260'})[0].get_text(strip=True)

In [15]:
# DataFrame с собранной информацией
df = pd.DataFrame(columns=[
    'Тема обращения',
    'Текст обращения',
    'Время обращения',
    'Имя пользователя - автора обращения',
    'Регион составителя обращения',
    'Текст ответа',
    'Время ответа',
    'Имя пользователя - автора ответа',
    'Оценка клиента'
])

# перебираем все оценки 1-5
for rate in range(1, 6):
    # перебираем номера страниц 1-3
    for page_number in range(1, 4):
        url = 'https://www.banki.ru/services/responses/bank/bnpparibaseast/?rate=' + str(rate) + '&is_countable=on&page=' + str(page_number) + '&isMobile=0'

        r = requests.get(url)
        s = BeautifulSoup(r.text, 'lxml')

        appeal_numbers = re.findall('/services/responses/bank/response/(\d*)', r.text)
        appeal_numbers = list(dict.fromkeys(appeal_numbers))

        for appeal_number in appeal_numbers:
            r_appeal = requests.get(f'https://www.banki.ru/services/responses/bank/response/{appeal_number}/')
            s_appeal = BeautifulSoup(r_appeal.text, 'lxml')


            df.loc[len(df)] = [
                get_theme_appeal(s_appeal),
                get_text_appeal(s_appeal),
                get_time_appeal(s_appeal),
                get_user_appeal(s_appeal),
                get_por_appeal(s_appeal),
                get_text_reply(s_appeal),
                get_time_reply(s_appeal),
                get_user_reply(s_appeal),
                rate
            ]
            
            # sleep для избегания перегрузки сайта запросами в единицу времени
            time.sleep(random.random() + 1)

        print(f'Обработана страница {page_number} отзывов с оценкой {rate}')

Обработана страница 1 отзывов с оценкой 1
Обработана страница 2 отзывов с оценкой 1
Обработана страница 3 отзывов с оценкой 1
Обработана страница 1 отзывов с оценкой 2
Обработана страница 2 отзывов с оценкой 2
Обработана страница 3 отзывов с оценкой 2
Обработана страница 1 отзывов с оценкой 3
Обработана страница 2 отзывов с оценкой 3
Обработана страница 3 отзывов с оценкой 3
Обработана страница 1 отзывов с оценкой 4
Обработана страница 2 отзывов с оценкой 4
Обработана страница 3 отзывов с оценкой 4
Обработана страница 1 отзывов с оценкой 5
Обработана страница 2 отзывов с оценкой 5
Обработана страница 3 отзывов с оценкой 5


In [16]:
df

,Тема обращения,Текст обращения,Время обращения,Имя пользователя - автора обращения,Регион составителя обращения,Текст ответа,Время ответа,Имя пользователя - автора ответа,Оценка клинета
0,Неправомерное начисление штрафа,12.10.2022 заключен автокредит №04******8414.2...,04.11.2022 11:58,user-486011841525,Санкт-Петербург,Добрый день!Согласно п. 12 Индивидуальных усло...,2022-11-07 08:39:51,Сетелем Банк,1
1,Ода о страховке,"Доброго времени суток, уважаемые мои.Хочу пове...",28.10.2022 18:37,user-698610903546,Москва,Уважаемый Клиент!Мы внимательно изучили Вашу с...,2022-10-31 15:58:25,Сетелем Банк,1
2,Штрафы за непредоставление СТС,Добрый день!Моя история не нова. Автокредит 04...,19.09.2022 11:46,cyxapuk.,Санкт-Петербург,Спасибо за отзыв. Давайте подождем от банка ре...,2022-09-23 14:12:15,Администратор народного рейтинга,1
3,Ужасное обслуживание. Необоснованные штрафы,"Автодилер при продаже автомобиля, предложил во...",16.09.2022 13:54,user-35309910387,Санкт-Петербург,Спасибо за отзыв. Давайте подождём ответа банк...,2022-09-21 16:06:00,Администратор народного рейтинга,1
4,Штрафные санкции,9.08.22 оформлен кредит на покупку автомобиля ...,02.09.2022 11:28,user-37369743590,Новосибирск (Новосибирская область),"Оценку зачли. Если банк решил вашу проблему, в...",2022-09-08 17:31:16,Администратор народного рейтинга,1
...,...,...,...,...,...,...,...,...,...
370,Досрочное погашение,"Хитрая система Сетелем Банка, описываю:1) Хочу...",10.02.2021 14:55,oh_yeah,Санкт-Петербург,Оценка изменена по просьбе автора.,2021-02-16 15:42:36,Администратор народного рейтинга,5
371,Досрочное погашение кредита,Сотрудники отказывают в досрочном погашении кр...,12.01.2021 13:55,Александра36,Воронеж (Воронежская область),Спасибо за отзыв. Как вы можете прокомментиров...,2021-01-21 10:19:43,Администратор народного рейтинга,5
372,Автокредит,Ставлю оценку пять!За то что когда я обратилас...,17.12.2020 16:08,Cdsolga,Москва,Добрый день.Спасибо за положительный отзыв! В ...,2020-12-18 11:12:41,Сетелем Банк,5
373,Отзыв о работе банка и сотрудниках,Добрый день!После заключения Договора потребит...,16.12.2020 10:42,130956zz,Екатеринбург (Свердловская область),"Добрый день.Приятно, что работа сотрудников Ба...",2020-12-16 11:38:57,Сетелем Банк,5


In [17]:
df.to_csv('output/df_appeals.csv', index=False)

---
<a name="4"></a>

## Реализация с selenium

In [ ]:
# DataFrame с собранной информацией
df_selenium = pd.DataFrame(columns=[
    'Тема обращения',
    'Текст обращения',
    'Время обращения',
    'Имя пользователя - автора обращения',
    'Регион составителя обращения',
    'Текст ответа',
    'Время ответа',
    'Имя пользователя - автора ответа',
    'Оценка клиента'
])

driver = webdriver.Chrome(executable_path='selenium_chromedriver/chromedriver')

# перебираем все оценки 1-5
for rate in range(1, 6):
    # перебираем номера страниц 1-3
    for page_number in range(1, 2):
        url = 'https://www.banki.ru/services/responses/bank/bnpparibaseast/?rate=' + str(rate) + '&is_countable=on&page=' + str(page_number) + '&isMobile=0'

        driver.get(url)
        html_source_main = driver.page_source
        appeal_numbers = re.findall('/services/responses/bank/response/(\d*)', html_source_main)
        appeal_numbers = list(dict.fromkeys(appeal_numbers))

        for appeal_number in appeal_numbers:
            driver.get(f'https://www.banki.ru/services/responses/bank/response/{appeal_number}/')
            html_source_appeal = driver.page_source
            s_appeal = BeautifulSoup(html_source_appeal, 'lxml')

            df_selenium.loc[len(df_selenium)] = [
                get_theme_appeal(s_appeal),
                get_text_appeal(s_appeal),
                get_time_appeal(s_appeal),
                get_user_appeal(s_appeal),
                get_por_appeal(s_appeal),
                get_text_reply(s_appeal),
                get_time_reply(s_appeal),
                get_user_reply(s_appeal),
                rate
            ]
            
            # sleep для избегания перегрузки сайта запросами в единицу времени
            # time.sleep(random.random() + 1)

        print(f'Обработана страница {page_number} отзывов с оценкой {rate}')

In [ ]:
df_selenium

In [ ]:
df_selenium.to_csv('output/df_selenium_appeals.csv', index=False)

---
<a name="5"></a>

## Анализ данных

In [6]:
# посчитаем статистику времени ответа на обращение
df_raw = pd.read_csv('output/df_appeals.csv')
df = df_raw[['Время обращения', 'Время ответа']].copy()
df = df.rename(columns={'Время обращения': 't1', 'Время ответа': 't2'})
df

,t1,t2
0,04.11.2022 11:58,2022-11-07 08:39:51
1,28.10.2022 18:37,2022-10-31 15:58:25
2,19.09.2022 11:46,2022-09-23 14:12:15
3,16.09.2022 13:54,2022-09-21 16:06:00
4,02.09.2022 11:28,2022-09-08 17:31:16
...,...,...
370,10.02.2021 14:55,2021-02-16 15:42:36
371,12.01.2021 13:55,2021-01-21 10:19:43
372,17.12.2020 16:08,2020-12-18 11:12:41
373,16.12.2020 10:42,2020-12-16 11:38:57


In [7]:
df['t1'] = pd.to_datetime(df['t1'], dayfirst=True)
df['t2'] = pd.to_datetime(df['t2'])
df['t2-t1'] = (df['t2'] - df['t1']).dt.total_seconds()/3600
df

,t1,t2,t2-t1
0,2022-11-04 11:58:00,2022-11-07 08:39:51,68.697500
1,2022-10-28 18:37:00,2022-10-31 15:58:25,69.356944
2,2022-09-19 11:46:00,2022-09-23 14:12:15,98.437500
3,2022-09-16 13:54:00,2022-09-21 16:06:00,122.200000
4,2022-09-02 11:28:00,2022-09-08 17:31:16,150.054444
...,...,...,...
370,2021-02-10 14:55:00,2021-02-16 15:42:36,144.793333
371,2021-01-12 13:55:00,2021-01-21 10:19:43,212.411944
372,2020-12-17 16:08:00,2020-12-18 11:12:41,19.078056
373,2020-12-16 10:42:00,2020-12-16 11:38:57,0.949167


In [8]:
# видим одно обращение без определения времени ответа
df.sort_values('t2-t1')

,t1,t2,t2-t1
306,2022-10-04 19:12:00,2022-10-04 19:16:21,0.072500
141,2018-06-22 20:49:00,2018-06-22 21:00:47,0.196389
236,2021-06-11 14:30:00,2021-06-11 14:43:46,0.229444
283,2019-02-20 16:13:00,2019-02-20 16:27:35,0.243056
373,2020-12-16 10:42:00,2020-12-16 11:38:57,0.949167
...,...,...,...
209,2014-08-19 00:58:00,2014-10-28 09:14:00,1688.266667
332,2022-04-08 15:00:00,2022-06-20 17:27:48,1754.463333
7,2022-07-09 01:33:00,2022-09-29 15:41:00,1982.133333
27,2022-01-28 12:02:00,2022-04-27 11:43:28,2135.691111


In [9]:
# действительно нет теккста и времени ответа
df_raw.loc[341]

Тема обращения                                                               Просто вау!
Текст обращения                        Никогда не писал отзывы но здесь захотел, пото...
Время обращения                                                         26.11.2021 12:52
Имя пользователя - автора обращения                                        Pavlushka1992
Регион составителя обращения                                                      Москва
Текст ответа                                                                         н/д
Время ответа                                                                         NaN
Имя пользователя - автора ответа                                           Pavlushka1992
Оценка клинета                                                                         5
Name: 341, dtype: object

In [10]:
# выкинем эту одну запись без времени ответа
df = df.dropna()
# статистика по времени между ответом и обращением
df.describe()

,t2-t1
count,374.000000
mean,165.108734
std,261.172038
min,0.072500
25%,39.457708
50%,99.419306
75%,163.429444
max,2135.691111
